In [2]:
'''
Project : Udacity project_showcase_challenge
Team    : Sandeep Pawar, Aarthi Alagammai
Version : 1.2
Date    : 9/11/2020

'''

'\nProject : Udacity project_showcase_challenge\nTeam    : Sandeep Pawar, Aarthi Alagammai\nVersion : 1.2\nDate    : 9/11/2020\n\n'

# Import packages 

In [1]:
import numpy as np
import pandas as pd
import os
import azureml.core
import forecast

from azureml.core import Workspace

ws = Workspace.from_config()

# Model Registration


In [3]:


from azureml.core.model import Model

model = Model.register(model_path="forecast.py",
                       model_name="Fuel_Sales_Forecast",
                       tags={'area': "fuel_sales", 'type': "classical_forecast"},
                       description="Forecast the three fuels",
                       workspace=ws)

MODEL_DIR  = "..\forecasting\model\ "

Registering model Fuel_Sales_Forecast


# Enviroment Creation

In [4]:
from azureml.core.environment import Environment

packages = ['pandas', 'statsmodels', 'scipy', 'fbprophet', 'numpy', "inference-schema" ]

environment = Environment("LocalDeploy")
for package in packages:
    environment.python.conda_dependencies.add_pip_package(package)

print(environment)



Environment(Name: LocalDeploy,
Version: None)


# Scoring

In [5]:


%%writefile forecast_score.py


from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


def init():

    global model
    
    model = os.path.join(os.getenv('MODEL_DIR'), 'forecast.py')
    

    input_sample = pd.DataFrame({'Fuel':,['a'] * 10
                                 'Sales': np.arange(1,11, dtype='float64').tolist()}, 
                                  index=pd.date_range('1-1-2000', freq='MS',periods=10))


    output_sample = pd.DataFrame({'Pellet'    : np.arange(1,11, dtype='float64').tolist(), 
                                  'Firewood'  : np.arange(1,11, dtype='float64').tolist(),
                                  'Briquette' : np.arange(1,11, dtype='float64').tolist()})
    
    

@input_schema('data', PandasParameterType(input_sample, enforce_shape=False))
@output_schema(PandasParameterType (output_sample))

data_path   = "https://powerbibyos3.blob.core.windows.net/powerbi/Trial/test/Query.csv.snapshots/data.csv"
result_path = "https://powerbibyos3.blob.core.windows.net/powerbi/Trial/result/"



def run(data):
    try:
    
        
        result = pd.DataFrame({ 
                                 'Pellet': pellet_forecast(data.query('Fuel == "Pellet"')['Sales'],12).tolist(),
                                 'Firewood': firewood_forecast(data.query('Fuel == "Firewood"')['Sales'],12).tolist(),
                                 'Briquette': briquette_forecast(data.query('Fuel == "Briquette"')['Sales'],12).tolist()
                             })
        
        return result.to_csv(result_path, sep='\t')
    
    except Exception as e:
        error = str(e)
        return error

Writing forecast_score.py


# Create Inference Configuration

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script=forecat_score.py,
                                   environment=environment)

# Deploy Local Web Service

In [ ]:
from azureml.core.webservice import LocalWebservice

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=5885)

local_service = Model.deploy(ws, "inference", model , inference_config, deployment_config)

local_service.wait_for_deployment()

# Run Inference

In [ ]:

data = (
           import_data(data_path)
           .pipe(clean_data)
       ) 


result = pd.DataFrame({ 
                             'Pellet'   : pellet_forecast(data[data.Fuel=='Pellets']['Sales'].sort_index(),12).tolist(),
                             'Firewood' : firewood_forecast(data[data.Fuel=='Pellets']['Sales'].sort_index(),12).tolist(),
                             'Briquette': briquette_forecast(data[data.Fuel=='Pellets']['Sales'].sort_index(),12).tolist()
                   
                     })
        
local_service.run(data)